In [ ]:
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
from models.common import Conv

: 

In [ ]:

# ==========================================
# [설정]
MODEL_PATH = "runs/train/hanja_1to10/weights/best.pt" # 학습된 모델 경로
PRUNING_AMOUNT = 0.3  # 30%를 잘라냄 (0.3 ~ 0.5 추천)
# ==========================================

def main():
    print(f"✂️ Pruning 시작: {MODEL_PATH}")
    print(f"✂️ 목표: 전체 가중치의 {PRUNING_AMOUNT*100}% 제거")

    # 1. 모델 로드
    # (YOLOv5 내부 코드를 사용하므로 yolov5 폴더 안에서 실행하거나 경로 설정 필요)
    ckpt = torch.load(MODEL_PATH, map_location='cpu')
    model = ckpt['model'].float()

    # 2. Pruning 적용 (모든 Conv2d 레이어 대상)
    # YOLOv5의 핵심인 Conv 레이어를 찾아서 가중치 중 작은값 하위 30%를 0으로 만듦
    count = 0
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            # L1 Norm(절댓값) 기준으로 가지치기 (notebook 내용과 동일)
            prune.l1_unstructured(module, name='weight', amount=PRUNING_AMOUNT)
            
            # 가지치기 "영구 적용" (mask를 weight에 아예 반영해버림)
            prune.remove(module, 'weight')
            count += 1

    print(f"✅ 총 {count}개의 Conv2d 레이어에 Pruning 적용 완료!")

    # 3. 저장
    # 구조를 유지한 채로 가중치만 0이 된 상태로 저장
    ckpt['model'] = model
    
    # Pruned 모델 저장
    save_path = MODEL_PATH.replace('.pt', '_pruned.pt')
    torch.save(ckpt, save_path)
    
    print(f"💾 Pruning된 모델 저장 완료: {save_path}")
    print("👉 이 모델을 다시 ONNX -> TensorRT로 변환해서 사용하세요.")
    print("⚠️ 주의: Pruning 후에는 정확도가 떨어질 수 있으므로, 가능하다면 재학습(Fine-tuning)을 1~2 에포크 돌려주는 것이 좋습니다.")

if __name__ == "__main__":
    main()